# Protein conformational changes between conditions


## Getting the data

In [14]:
!pip install Bio

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
from sklearn.model_selection import train_test_split

In [16]:
from google.colab import drive
import os

gdrive_path='/content/gdrive/MyDrive/UFSC/Faculdade/tum/Genomics'

# This will mount your google drive under 'MyDrive'
drive.mount('/content/gdrive', force_remount=True)
# In order to access the files in this notebook we have to navigate to the correct folder
os.chdir(gdrive_path)
# Check manually if all files are present
print(sorted(os.listdir()))

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import pandas as pd

df = pd.read_excel('OsmoticStress.xlsx')

In [ ]:
df = df[['Uniprot_ID', 'Peptide_sequence', 'Log2FC(LiP_norm)']]

In [ ]:
df.head()

In [ ]:
from Bio import SeqIO

def read_fasta_to_dict(file_path):
    sequences = {}
    for record in SeqIO.parse(file_path, "fasta"):
        # Splitting the record.id to extract the UniProt ID if necessary
        uniprot_id = record.id.split('|')[1]  # Adjust this as per your FASTA file's format
        sequences[uniprot_id] = str(record.seq)
    return sequences

# Provide the path to your FASTA file
fasta_file_path = 'UP000002311_559292.fasta'
sequences = read_fasta_to_dict(fasta_file_path)

In [ ]:
df["full_sequence"] = df['Uniprot_ID'].map(sequences)

In [ ]:
df.head()

In [ ]:
df.to_csv(gdrive_path + "/processed_yeast_df.csv")

In [ ]:
#TODO Split training and testing data

## Getting the model from hugging face


In [ ]:
from huggingface_hub import notebook_login

notebook_login()


Choose one of the following checkpoints:


| Checkpoint name | Num layers | Num parameters |
|------------------------------|----|----------|
| `esm2_t48_15B_UR50D`         | 48 | 15B     |
| `esm2_t36_3B_UR50D`          | 36 | 3B      |
| `esm2_t33_650M_UR50D`        | 33 | 650M    |
| `esm2_t30_150M_UR50D`        | 30 | 150M    |
| `esm2_t12_35M_UR50D`         | 12 | 35M     |
| `esm2_t6_8M_UR50D`           | 6  | 8M      |

In [ ]:
model_checkpoint = "facebook/esm2_t6_8M_UR50D"

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
LiP_norm = df['Log2FC(LiP_norm)']
tokenizer_train, tokenizer_test, LiP_norm_train, LiP_norm_test = train_test_split(tokenizer, LiP_norm, test_size = 0.25)